# Описание гипотезы
### Проверим предположения
1. Определить, насколько обучение сокращает время прохождения этапов игры.
2. Доказать, что успешное обучение само по себе влияет на оплату и не имеет значения то, каким этапом оно шло.
3. Проверить, насколько прозрачен процесс взаимодействия с игрой.
#### Проверять будем на данных пользователей, зарегистрированных в 2018 году (1 января 2018 - 31 декабря 2018)

Импротируем нужные библиотеки

In [16]:
import pandas as pd
import numpy as np

Загружаем данные

In [17]:
events_df = pd.read_csv('data/7_4_Events.csv', sep=',')
purchase_df = pd.read_csv('data/purchase.csv', sep=',')

## Производим предобработку данных

Выделяем пользователей зарегистрирванных в 2018 году и загружаем данные по ним в отдельные датафреймы

In [18]:
# Создаем фильтр по пользователям, зарегистрировавшихся в 2018 году
mask = (events_df['start_time'] >= '2018-01-01') & (events_df['start_time'] < '2019-01-01') & (events_df['event_type'] == 'registration')

regist_2018 = events_df[mask]['user_id'].to_list() # список пользователей зарегистрировавшихся в 2018 году 

events_2018 = events_df[events_df['user_id'].isin(regist_2018)].copy() # отбираем все записи с этими пользователями в таблице events
purchase_2018 = purchase_df[purchase_df['user_id'].isin(regist_2018)].copy() # отбираем все записи с этими пользователями в таблице purchase


Преобразуем даты в полученых датафреймах

In [19]:
purchase_2018['event_datetime'] = pd.to_datetime(purchase_2018['event_datetime'])
events_2018['start_time'] = pd.to_datetime(events_2018['start_time'])

Перед объединением датафреймов, переименуем колонки идентификаторов

In [20]:
events_2018 = events_2018.rename(columns={'id':'event_id'})
purchase_2018 = purchase_2018.rename(columns={'id':'purchase_id'})
purchase_2018['event_type'] = 'purchase'

Сделаем объединенный датафрейм из событий и оплат

In [22]:
total_events_df = pd.concat([events_2018,purchase_2018],sort=False)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

### Проверим 1-ю гипотезу
* Насколько обучение сокращает время прохождения этапов игры

Сформируем группы пользователей, которые проходили обучение и которые не проходили

In [23]:
# Пользователи которые начали обучение
users_tutorial_start = total_events_df[total_events_df['event_type'] == 'tutorial_start']['user_id'].unique()
# Пользователи которые прошли обучение
users_tutorial_finish = total_events_df[total_events_df['event_type'] == 'tutorial_finish']['user_id'].unique()
# Пользователт которые закончили обучение, по любому его начинали, а пользователи которые начали обучение,
# могли его не закончить, убираем из первого списка пользователей, которые не закончили обучение
users_tutorial_start = users_tutorial_start[~np.isin(users_tutorial_start, users_tutorial_finish)]
# Пользователи которые вообще не проходили обучение
users_not_tutorial = total_events_df[~(total_events_df['user_id'].isin(users_tutorial_finish) | total_events_df['user_id'].isin(users_tutorial_start))]['user_id'].unique()

Создаем список групп

In [24]:
users_group = [
    {'tut_start': users_tutorial_start},
    {'tut_finish': users_tutorial_finish},
    {'tut_no': users_not_tutorial}
]

Нас интересует время прохождения уровней, создадим вспомогательный датафрейм с выбором уровней

In [27]:
level_choise_total = total_events_df[total_events_df['event_type']=='level_choice']

Пройдемся по группам и соберем нужную информацию

In [ ]:
for group in users_group:
    tutorial = list(group.keys())[0] # группа
    print('Сводка по уровню:', tutorial)
    group_users = group[tutorial] # датафрейм группы
    count_of_users_in_group = len(group_users) # количество пользователей в группе

In [26]:
total_events_df.event_type.unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice', 'purchase'], dtype=object)

In [12]:
print(len(users_not_tutorial))
print(len(users_tutorial_finish))
print(len(users_tutorial_start))
print(len(total_events_df['user_id'].unique()))

8068
10250
1608
19926


In [28]:
level_choise_total

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
19,80327.0,level_choice,medium,2018-01-01 20:37:22,NaN,27835,NaN,NaT,NaN
23,80331.0,level_choice,hard,2018-01-01 22:37:50,NaN,27839,NaN,NaT,NaN
36,80344.0,level_choice,medium,2018-01-02 05:18:42,NaN,27840,NaN,NaT,NaN
38,80346.0,level_choice,hard,2018-01-02 06:19:18,NaN,27845,NaN,NaT,NaN
45,80353.0,level_choice,easy,2018-01-02 08:46:03,NaN,27842,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
66932,147240.0,level_choice,medium,2018-12-31 09:59:00,NaN,47732,NaN,NaT,NaN
66937,147245.0,level_choice,medium,2018-12-31 12:24:40,NaN,47747,NaN,NaT,NaN
66948,147256.0,level_choice,medium,2018-12-31 22:36:19,NaN,47753,NaN,NaT,NaN
66956,147265.0,level_choice,easy,2019-01-01 05:04:52,NaN,47755,NaN,NaT,NaN
